# Artificial Neural Network

In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from common import *

In [2]:
if not os.path.exists('data.csv'):
    if not os.path.exists('Fake.csv') and not os.path.exsits('True.csv'):
        download_data()
    label_data('Fake.csv', 'fake_labeled.csv', 1)
    label_data('True.csv', 'true_labeled.csv', 0)
    os.system("awk '(NR == 1) || (FNR > 1)' *labeled.csv > data.csv")

In [12]:
df = pd.read_csv('data.csv', usecols=['text', 'label'], encoding='utf-8')
df = df.sample(n=NUM_SAMPLES, random_state=RANDOM_SEED)

In [15]:
countVectorizer = CountVectorizer()
embeddings = preprocess_data(df['text'], countVectorizer)

X_tr, X_ts, y_tr, y_ts= train_test_split(embeddings, df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

X_tr = X_tr.toarray()
X_ts = X_ts.toarray()

print(f'Train shape: {X_tr.shape}')
print(f'Test shape: {X_ts.shape}')

Train shape: (1500, 49912)
Test shape: (1500, 49912)


In [16]:
#first basic ann
from keras.models import Sequential
from keras import layers

input_dim = X_tr.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

2022-04-19 17:02:06.413389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 17:02:06.413414: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                499130    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 499,141
Trainable params: 499,141
Non-trainable params: 0
_________________________________________________________________


2022-04-19 17:02:08.485033: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-19 17:02:08.485064: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-19 17:02:08.485111: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-04-19 17:02:08.485308: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
history = model.fit(
    X_tr, y_tr,
    epochs=EPOCHS,
    verbose=True,
    validation_data=(X_ts, y_ts),
    batch_size=10
)

2022-04-19 17:02:14.408551: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


Epoch 1/10
143/150 [===========================>..] - ETA: 0s - loss: 0.3161 - accuracy: 0.8951

2022-04-19 17:02:16.905171: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


150/150 [==============================] - 3s 16ms/step - loss: 0.3087 - accuracy: 0.8993 - val_loss: 0.1599 - val_accuracy: 0.9680
Epoch 2/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0614 - accuracy: 0.9980 - val_loss: 0.1350 - val_accuracy: 0.9727
Epoch 3/10
150/150 [==============================] - 2s 10ms/step - loss: 0.0307 - accuracy: 1.0000 - val_loss: 0.1242 - val_accuracy: 0.9747
Epoch 4/10
150/150 [==============================] - 2s 10ms/step - loss: 0.0208 - accuracy: 1.0000 - val_loss: 0.1257 - val_accuracy: 0.9733
Epoch 5/10
150/150 [==============================] - 2s 10ms/step - loss: 0.0157 - accuracy: 1.0000 - val_loss: 0.1252 - val_accuracy: 0.9733
Epoch 6/10
150/150 [==============================] - 1s 9ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 0.1277 - val_accuracy: 0.9727
Epoch 7/10
150/150 [==============================] - 1s 9ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.1303 - val_accuracy: 0.9727
Epoch 8/10
1

In [19]:
from keras.backend import clear_session
clear_session()

loss, accuracy = model.evaluate(X_tr, y_tr, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_ts, y_ts, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

import matplotlib.pyplot as plt
plt.style.use('ggplot')

Training Accuracy: 1.0000
Testing Accuracy:  0.9713


# CNN

# BNN